Importation des packages et modules

In [878]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector

Lecture du dataset en .csv avec Pandas, enregistré en DataFrame Pandas

In [879]:
df = pd.read_csv(r"Data\Raw\student-depression-dataset.csv")

Affichage des 5 premières lignes du DataFrame Pandas

In [880]:
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


Shape du DataFrame Pandas

In [881]:
df.shape

(27901, 18)

Size du DataFrame Pandas

In [882]:
df.size

502218

Types des éléments du DataFrame :

In [883]:
df.dtypes

id                                         int64
Gender                                    object
Age                                      float64
City                                      object
Profession                                object
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                            object
Dietary Habits                            object
Degree                                    object
Have you ever had suicidal thoughts ?     object
Work/Study Hours                         float64
Financial Stress                         float64
Family History of Mental Illness          object
Depression                                 int64
dtype: object

Index du DataFrame Pandas

In [884]:
df.index

RangeIndex(start=0, stop=27901, step=1)

Colonnes du DataFrame Pandas

In [885]:
df.columns

Index(['id', 'Gender', 'Age', 'City', 'Profession', 'Academic Pressure',
       'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction',
       'Sleep Duration', 'Dietary Habits', 'Degree',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')

Comptage du nb de NaN dans le DataFrame

In [886]:
df.isna().sum(axis=0)

id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         3
Family History of Mental Illness         0
Depression                               0
dtype: int64

Trouver NaN avec Boolean Mask

In [887]:
nan_indices = df[df["Financial Stress"].isna()].index
nan_indices

Index([4458, 13596, 19266], dtype='int64')

Supprimer toutes les lignes contenant au moins un NaN

In [888]:
df_nona = df.dropna()

In [889]:
df_nona.shape

(27898, 18)

In [890]:
df_nona.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


Dictionnaires des variables catégorielles et nombre d'occurences

In [891]:
# Sélectionner les colonnes catégorielles
categorical_features = df_nona.select_dtypes(include=['object']).columns

# Dictionnaire pour stocker les occurrences des catégories pour chaque variable catégorielle
category_occurrences = {}

# Pour chaque colonne catégorielle
for feature in categorical_features:
    # Compter les occurrences de chaque valeur dans la colonne
    occurrences = df_nona[feature].value_counts().to_dict()
    # Ajouter le résultat au dictionnaire global
    category_occurrences[feature] = occurrences
    # Afficher le dictionnaire des occurrences pour chaque colonne
    print(f"{feature} : {occurrences}")


Gender : {'Male': 15546, 'Female': 12352}
City : {'Kalyan': 1570, 'Srinagar': 1372, 'Hyderabad': 1339, 'Vasai-Virar': 1290, 'Lucknow': 1155, 'Thane': 1139, 'Ludhiana': 1111, 'Agra': 1094, 'Surat': 1078, 'Kolkata': 1065, 'Jaipur': 1036, 'Patna': 1007, 'Visakhapatnam': 969, 'Pune': 968, 'Ahmedabad': 951, 'Bhopal': 934, 'Chennai': 885, 'Meerut': 825, 'Rajkot': 816, 'Delhi': 768, 'Bangalore': 767, 'Ghaziabad': 745, 'Mumbai': 699, 'Vadodara': 694, 'Varanasi': 684, 'Nagpur': 651, 'Indore': 643, 'Kanpur': 609, 'Nashik': 547, 'Faridabad': 461, 'Saanvi': 2, 'Bhavna': 2, 'City': 2, 'Harsha': 2, 'Kibara': 1, 'Nandini': 1, 'Nalini': 1, 'Mihir': 1, 'Nalyan': 1, 'M.Com': 1, 'ME': 1, 'Rashi': 1, 'Gaurav': 1, 'Reyansh': 1, 'Harsh': 1, 'Vaanya': 1, 'Mira': 1, 'Less than 5 Kalyan': 1, '3.0': 1, 'Less Delhi': 1, 'M.Tech': 1, 'Khaziabad': 1}
Profession : {'Student': 27867, 'Architect': 8, 'Teacher': 6, 'Digital Marketer': 3, 'Content Writer': 2, 'Chef': 2, 'Doctor': 2, 'Pharmacist': 2, 'Civil Engineer': 1

Suppression des entrées associées à des valeurs uniques ou négligeables de variable catégorielle

In [892]:
# Filtrage des entrées selon les critères fournis

# Filtrer la colonne "Profession" pour garder seulement "Student"
df_cat_filtered = df_nona[df_nona['Profession'] == 'Student']

# Filtrer la colonne "City" pour garder les villes ayant plus de 10 occurrences
city_counts = df_cat_filtered['City'].value_counts()
cities_to_keep = city_counts[city_counts >= 10].index
df_cat_filtered = df_cat_filtered[df_cat_filtered['City'].isin(cities_to_keep)]

# Filtrer la colonne "Sleep Duration" pour enlever "Others"
df_cat_filtered = df_cat_filtered[df_cat_filtered['Sleep Duration'] != 'Others']

# Filtrer la colonne "Dietary Habits" pour enlever "Others"
df_cat_filtered = df_cat_filtered[df_cat_filtered['Dietary Habits'] != 'Others']

# Filtrer la colonne "Degree" pour enlever "Others"
df_cat_filtered = df_cat_filtered[df_cat_filtered['Degree'] != 'Others']

# Afficher le résultat final après filtrage
print(f"Nombre d'entrées après filtrage: {df_cat_filtered.shape[0]}")


Nombre d'entrées après filtrage: 27776


Suppression Feature "Profession" car constante "Student" sur toutes les entrées

In [893]:
df_cat_filtered = df_cat_filtered.drop("Profession", axis=1)

Vérification constante ou tendance marquée pour variables numériques (intuition : "Job Satisfaction" constante égale à 0 car uniquement des étudiants restants dans le DataFrame suite au filtrage)

In [894]:
# Sélectionner les colonnes catégorielles
numerical_features = df_cat_filtered.select_dtypes(include=np.number).columns

# Dictionnaire pour stocker les occurrences des catégories pour chaque variable catégorielle
category_occurrences = {}

# Pour chaque colonne catégorielle
for feature in numerical_features:
    # Compter les occurrences de chaque valeur dans la colonne
    occurrences = df_cat_filtered[feature].value_counts().to_dict()
    # Ajouter le résultat au dictionnaire global
    category_occurrences[feature] = occurrences
    # Afficher le dictionnaire des occurrences pour chaque colonne
    print(f"{feature} : {occurrences}")

id : {2: 1, 94223: 1, 94276: 1, 94275: 1, 94272: 1, 94270: 1, 94256: 1, 94236: 1, 94235: 1, 94234: 1, 94231: 1, 94230: 1, 94227: 1, 94219: 1, 94448: 1, 94204: 1, 94202: 1, 94197: 1, 94192: 1, 94184: 1, 94183: 1, 94174: 1, 94173: 1, 94172: 1, 94165: 1, 94162: 1, 94280: 1, 94289: 1, 94292: 1, 94304: 1, 94435: 1, 94429: 1, 94427: 1, 94425: 1, 94420: 1, 94407: 1, 94400: 1, 94397: 1, 94396: 1, 94393: 1, 94386: 1, 94373: 1, 94370: 1, 94369: 1, 94362: 1, 94358: 1, 94354: 1, 94351: 1, 94347: 1, 94325: 1, 94320: 1, 94312: 1, 94307: 1, 94156: 1, 94147: 1, 94133: 1, 93988: 1, 93985: 1, 93983: 1, 93975: 1, 93969: 1, 93962: 1, 93954: 1, 93946: 1, 93936: 1, 93934: 1, 93931: 1, 93930: 1, 93922: 1, 93916: 1, 93915: 1, 93912: 1, 93909: 1, 93906: 1, 93901: 1, 93899: 1, 93898: 1, 93871: 1, 93868: 1, 93862: 1, 93986: 1, 93996: 1, 94131: 1, 94001: 1, 94128: 1, 94121: 1, 94118: 1, 94116: 1, 94113: 1, 94111: 1, 94100: 1, 94093: 1, 94079: 1, 94077: 1, 94073: 1, 94071: 1, 94061: 1, 94060: 1, 94056: 1, 94050: 1

Ce qui attire l'attention, les variables ou il y a des valeurs très minoritaires : "Work Pressure" valeurs égale à 5.0 et 2.0 car que des étudiants, idem pour "Job Satisfaction". "Academic Pressure" où valeur égale à 0.0 et idem pour "Study Satisfaction". On va donc regarder les entrées associées en détail avant de filtrer.

In [895]:
# Filtrer les entrées selon les conditions spécifiées
work_pressure_non_zero = df_cat_filtered[df_cat_filtered['Work Pressure'] != 0.0]
job_satisfaction_non_zero = df_cat_filtered[df_cat_filtered['Job Satisfaction'] != 0.0]
academic_pressure_zero = df_cat_filtered[df_cat_filtered['Academic Pressure'] == 0.0]
study_satisfaction_zero = df_cat_filtered[df_cat_filtered['Study Satisfaction'] == 0.0]

# Afficher les résultats
print("Entrées où 'Work Pressure' est non égal à 0.0:")
print(work_pressure_non_zero)

print("\nEntrées où 'Job Satisfaction' est non égal à 0.0:")
print(job_satisfaction_non_zero)

print("\nEntrées où 'Academic Pressure' est égal à 0.0:")
print(academic_pressure_zero)

print("\nEntrées où 'Study Satisfaction' est égal à 0.0:")
print(study_satisfaction_zero)


Entrées où 'Work Pressure' est non égal à 0.0:
           id Gender  ...  Family History of Mental Illness Depression
4365    21880   Male  ...                                No          0
14855   75007   Male  ...                               Yes          0
25746  129756   Male  ...                                No          1

[3 rows x 17 columns]

Entrées où 'Job Satisfaction' est non égal à 0.0:
           id  Gender  ...  Family History of Mental Illness Depression
292      1428    Male  ...                                No          0
4365    21880    Male  ...                                No          0
13909   70453  Female  ...                               Yes          1
14855   75007    Male  ...                               Yes          0
20913  105773    Male  ...                                No          1
23141  116703    Male  ...                                No          1
25746  129756    Male  ...                                No          1
26719  134830  Fema

On observe que certaines entrées sont présentes dans les 4 tableaux. De plus toutes les autres anomalies sont sur des questions standardisées où il y a un gros écart de nombre d'occurrences avec les autres réponses possibles => réponse très minoritaire

In [896]:
# Filtrage des entrées selon les critères pour les variables numériques

# Filtrer la colonne "Work Pressure" pour garder les valeurs négales à 0.0
df_filtered = df_cat_filtered[df_cat_filtered['Work Pressure'] == 0.0]

# Filtrer la colonne "Job Satisfaction" pour garder les valeurs égales à 0.0
df_filtered = df_filtered[df_filtered['Job Satisfaction'] == 0.0]

# Filtrer la colonne "Academic Pressure" pour garder les valeurs non égales à 0.0
df_filtered = df_filtered[df_filtered['Academic Pressure'] != 0.0]

# Filtrer la colonne "Study Satisfaction" pour garder les valeurs non égales à 0.0
df_filtered = df_filtered[df_filtered['Study Satisfaction'] != 0.0]

# Afficher le résultat final après filtrage
print(f"Nombre d'entrées après filtrage pour les variables numériques: {df_filtered.shape[0]}")

Nombre d'entrées après filtrage pour les variables numériques: 27762


Distinction X (features) et y (target/label)

In [897]:
X = df_filtered.drop("Depression", axis=1)
y = df_filtered.loc[:, "Depression"]

PREPROCESSING

Encodage variables catégorielles


In [898]:
categorical_features = make_column_selector(dtype_include=object)  # Sélectionne les colonnes de type string/objet
cat_columns = categorical_features(X)  # Récupère les noms des colonnes catégorielles

# Initialiser l'encoder
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # `sparse=False` pour un DataFrame propre

# Transformer uniquement les colonnes catégorielles
encoded_cat = encoder.fit_transform(X[cat_columns])

# Créer un DataFrame avec les nouvelles colonnes encodées
encoded_cat_df = pd.DataFrame(encoded_cat, columns=encoder.get_feature_names_out(), index=X.index)

# Supprimer les anciennes colonnes catégorielles et concaténer les nouvelles
X_encoded = X.drop(columns=cat_columns)
X_encoded = pd.concat([X_encoded, encoded_cat_df], axis=1)

X_encoded.head()  # Vérifie le résultat

,id,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Gender_Female,Gender_Male,City_Agra,City_Ahmedabad,City_Bangalore,City_Bhopal,City_Chennai,City_Delhi,City_Faridabad,City_Ghaziabad,City_Hyderabad,City_Indore,City_Jaipur,City_Kalyan,City_Kanpur,City_Kolkata,City_Lucknow,City_Ludhiana,City_Meerut,City_Mumbai,City_Nagpur,City_Nashik,City_Patna,City_Pune,City_Rajkot,City_Srinagar,City_Surat,City_Thane,City_Vadodara,City_Varanasi,City_Vasai-Virar,City_Visakhapatnam,Sleep Duration_5-6 hours,Sleep Duration_7-8 hours,Sleep Duration_Less than 5 hours,Sleep Duration_More than 8 hours,Dietary Habits_Healthy,Dietary Habits_Moderate,Dietary Habits_Unhealthy,Degree_B.Arch,Degree_B.Com,Degree_B.Ed,Degree_B.Pharm,Degree_B.Tech,Degree_BA,Degree_BBA,Degree_BCA,Degree_BE,Degree_BHM,Degree_BSc,Degree_Class 12,Degree_LLB,Degree_LLM,Degree_M.Com,Degree_M.Ed,Degree_M.Pharm,Degree_M.Tech,Degree_MA,Degree_MBA,Degree_MBBS,Degree_MCA,Degree_MD,Degree_ME,Degree_MHM,Degree_MSc,Degree_PhD,Have you ever had suicidal thoughts ?_No,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_No,Family History of Mental Illness_Yes
0,2,33.0,5.0,0.0,8.97,2.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,8,24.0,2.0,0.0,5.90,5.0,0.0,3.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,26,31.0,3.0,0.0,7.03,5.0,0.0,9.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,30,28.0,3.0,0.0,5.59,2.0,0.0,4.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,32,25.0,4.0,0.0,8.13,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


Détection Outliers avec Isolation Forest

In [ ]:
model = IsolationForest(random_state=0, contamination=0.01)
model.fit(X_encoded)
model.predict(X_encoded) # tableau de -1 (anomalie) et de 1 (anomalie)

ValueError: could not convert string to float: 'Male'

Filtre du dataset avec Boolean Indexing

In [ ]:
outliers = model.predict(X_encoded) == -1
outliers

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,test_size=0.2)

Normalisation Robust Scaler au cas où Outliers

In [ ]:
from sklearn.preprocessing import RobustScaler

